In [1]:
import os

import utils

In [2]:
%%time

casename = "g.e22.G1850ECO_JRA_HR.TL319_t13.004"

# Directories to search for netCDF files
caseroot = os.path.join(os.sep, "glade", "work", "mlevy", "hi-res_BGC_JRA", "cases")
campaign_root = os.path.join(os.sep, "glade", "campaign", "cesm", "development", "bgcwg", "projects", "hi-res_JRA", "cases")
output_roots = [os.path.join(campaign_root, casename, "output")]
output_roots += utils.gen_output_roots_from_caseroot(os.path.join(caseroot, casename))

utils.timeseries_and_history_comparison(casename, output_roots)

Checking year 0001...
... checking stream pop.h.nyear1 ...
... checking stream pop.h.nday1 ...
... checking stream pop.h ...
... checking stream cice.h ...
All variables available in time series for year 0001
----
Checking year 0002...
... checking stream pop.h.nyear1 ...
... checking stream pop.h.nday1 ...
... checking stream pop.h ...
... checking stream cice.h1 ...
... checking stream cice.h ...
All variables available in time series for year 0002
----
Checking year 0003...
... checking stream pop.h.nyear1 ...
... checking stream pop.h.nday1 ...
... checking stream pop.h ...
... checking stream cice.h1 ...
... checking stream cice.h ...
All variables available in time series for year 0003
----
Checking year 0004...
... checking stream pop.h.nyear1 ...
... checking stream pop.h.nday1 ...
... checking stream pop.h ...
... checking stream cice.h1 ...
... checking stream cice.h ...
All variables available in time series for year 0004
----
Checking year 0005...
... checking stream pop.h.